In [41]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   environment.yml

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add .
!git commit -m "Add Pandas dep"

# Distinguishing tiles in one's pocket from tiles in the wild

Here's the challenge: Person X goes on a walk in the city, carrying a tile, and also a tile signal collector. X's tile rotates between a finite set of keys. X will obviously capture their own signal -- a certain number of *beacons* for each of the keys their tile is going through. They will also capture beacons from tiles in the pockets and bags of people they walk by. How can X distinguish the keys beaconed out of their tile from those of other people?

One would think that the sustained proximity between the person's own tile and their detector will make it so the largest **number of beacons collected** should be for their own keys. However, this is a poor decision variable, as it depends on the number of keys being rotated through, as well as the beacon frequency for the tile. Some brands of tiles might send beacons at much higher frequencies, making beacon numbers an imprecise appreciation of key ownership. A much better variable to test over should be **signal strength** (as measured in decibels): since X's own tile will always be closer to the detector than other people's, we should see an appreciably stronger signal from the former than from the latter.

Let's test this hypothesis from an early dataset we collected, where X's tile is an Apple AirTag that rotates between 4 keys.

---
## Accessing and loading the dataset

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import fsspec

In [ ]:
fs = fsspec.filesystem("abfs")
CONTAINER = "team1-3"
fs.ls(CONTAINER)

The files involved in this experiment are `April9DataOnly.txt`, which contains recorded beacons, and `April9MyAirtagRotatingKeys.txt`, which contains only beacons from X's tile. This last file is thus the ground truth: let's try and use the assumptions above to guess the 4 keys of X's tile.

What does the data file look like?

In [ ]:
def path_blob(path):
    return f"{CONTAINER}/{path}"

In [ ]:
fs.head(path_blob("April9DataOnly.txt"), 4096)

It's hard to eyeball, but this is a text file. On each line lives what looks like a Python dictionary. It looks like JSON too, but JSON is double-quoted, never single-quoted; we can't rely on Python's strict JSON decoders to help us here.

Fortunately, there is a safe way to interpret Python dictionary literals, using Python's embedded compiler front-end through the `ast` module.

In [ ]:
import ast

import pandas as pd
from tqdm.notebook import tqdm_notebook


def read_dict_stream(path):
    with fs.open(path_blob(path), mode="r", encoding="utf-8") as file:
        return pd.DataFrame([ast.literal_eval(line) for line in tqdm_notebook(file)])

In [ ]:
df = read_dict_stream("April9DataOnly.txt")
df

---

## From beacons to keys

So we have some 20k beacon records, for each of which we have signal strength written up in the `rssi` column. While a more thorough analysis of the semantics of this dataset would be useful, let's focus on our purpose for the time being. For each key, we want to capture aggregate statistics of the beacons we captured for it.

In [ ]:
keys_raw = df.groupby("manufacturerDataHex").agg({"rssi": ["count", "mean", "std", "min", "max"]})
keys_raw

So we have beacons for 1379 distinct keys. The number of beacons recorded for each key seems to vary quite a bit. We also see some variation in the dynamic range of the signal strength (the gap between minimal and maximal recorded signal strength). Let's take a look at the distribution of beacon counts.

In [ ]:
%matplotlib inline

In [ ]:
keys_raw[("rssi", "count")].hist(bins=[0, 10, 20, 50, 100, 200, 300, 400])

So for most of these keys (more than 1000 out of our 1379), we have 10 keys or less; these cannot possibly be X's keys. As flawed as this decision variable is, given that X has probably *not* been sticking their bodies very close to strangers' throughout their errand, we would still find their keys among those in some top-$N$ of the counts. Let's try the top-50 to start with.

In [ ]:
keys_top50 = keys_raw.sort_values(("rssi", "count"), ascending=False).head(50)
keys_top50

---

## Estimating average signal strength 

For each of these keys, we have between 111 and 372 beacons, each carrying its own measure of the signal strength. Thus, to assess a key's signal strength, we must proceed by *estimation*. It is not unreasonable to assume that our signal strength measures are *independant*. As such, the **average signal strength** associated to any key can be derived using the central limit theorem (involving Student's $t$ distribution as well as the mean and standard deviation), guiding us to build a confidence interval for each key. The following does this tersely, using a confidence level of 0.95.

In [ ]:
import numpy as np
import scipy.stats as stats

alpha = 1 - 0.95
ic = pd.DataFrame(
    np.outer(keys_top50[("rssi", "mean")], [1, 1]) + np.outer(
        (
            stats.t(df=keys_top50[("rssi", "count")] - 1).ppf(1 - alpha / 2)
            * keys_top50[("rssi", "std")]
            / np.sqrt(keys_top50[("rssi", "count")])
        ),
        [-1., 1.]
    ),
    index=keys_top50.index,
    columns=["lower", "upper"]
)
ic

Let's visualize these confidence intervals:

In [ ]:
import matplotlib.pyplot as plt

x = np.arange(len(keys_top50))
plt.plot(x, ic["lower"], 'bv', x, ic["upper"], 'b^')

I'm used to estimation jobs that involve tricky comparisons -- this one is *not tricky at all*. The assumption behind signal strength as a decision key for tile ownership is verified. We see that the keys for X's tile report signal strengths well above -50, while _every_ other one runs below this.

In [ ]:
plt.plot(x, ic["lower"], 'bv', x, ic["upper"], 'b^')
plt.plot(x, -50 * np.ones(x.shape), 'r--')

Let's thus use this as decision threshold.

---

## Have we found the keys?

In [ ]:
keys_x = keys_top50.loc[ic["lower"] > -50.]
answer = set(keys_x.index)
answer

Now, let's compare with the ground truth...

In [ ]:
ground_truth = set(read_dict_stream("April9MyAirtagRotatingKeys.txt").manufacturerDataHex)
if answer == ground_truth:
    print("🍰")

Yay, cake!

---

## The beacon count assumption

We had initially dismissed that the keys associated to the largest numbers of captured beacons would be ours. Let's compare:

In [ ]:
keys_x

These are emphatically not the largest-count keys we got. Actually, the last one comes in position 50, at the very bottom of the top-50 we analyzed. Yes, understanding that each key is beaconed out of the same tile, we get that X's tile is the one for which we have the most beacons. However, the split of X's beacon set between the keys is playing an important role in weakening beacon counts as ownership indicators. If somebody enlarges their tile's key set further, their per-key beacon counts will likely even drop out of the top quantiles, making this top-$N$ filtering actually impair the truth of their analysis. It's signal strength that decides, not beacon counts.

---

## Some extra-curricular work: the distribution of signal strength

Let's compare signal strength distribution for X's own 4 keys to those X captured in the wild.

In [ ]:
plt.figure(figsize=(15, 5))
for i, key in enumerate(answer, start=1):
    plt.subplot(1, 4, i)
    df.loc[df.manufacturerDataHex == key].rssi.hist(bins=20)

We observe that the assumption that drove our estimation of signal strength on the basis of the distribution being _central_ is flawed: the distribution of the best signals we've captured is actually skewed left, that would probably be better modeled using a Weibull distribution. It makes obvious that even when signal should be strong, it is _quite_ likely to get a baseline-weak measurement, whereas the signal strength is likely capped by some physical constraint on the good end.

What about the highest-count and lowest-count keys not belong to X?

In [ ]:
keys_most_beacons = [
    "4c00121900dee4cd376ddaf94db04e7dd3970e5560f90670b8a6840200",
    "4c0012190029f866de8e804ea0c8307ed030efb5f3e1895eff29130100",
    "4c001219007d891055838ac76a6e2d82a7e4019ad4d682cdf93b540100",
    "4c00121900fc3e5d40d6c9549bb10b2b6465c7cb90685cd44945ee0000"
]
keys_least_beacons = [
    "4c001219005d187ef77a26ac4bde0c17865cb2f1660d4634b605db0200",
    "4c00121900788e06e8affb2fa49f373c57d68f0e6d63e0918a37a90100",
    "4c00121900abf6fa47fa46f9a32c160f5152f124bb593e73442b5f0300",
    "4c00121900b761fee7b5eaacc0c74f77eeccb7de49015e9b98092e0000"
]
plt.figure(figsize=(15, 10))
for i, key in enumerate(keys_most_beacons + keys_least_beacons, start=1):
    plt.subplot(2, 4, i)
    df.loc[df.manufacturerDataHex == key].rssi.hist(bins=20)

For these significantly weaker signals, the centrality of the distribution tends to be quite better. The many-beacons cases of the top row show a less noisy picture than the less-beacons cases of the bottom row, which is a normal phenomenon bound to data quantity. So it seems that the leftward skew of the strong signals is effectively due to a natural device characteristic: we get the best possible signal strength in most cases, so all the variation comes on the lossy side.